In [5]:
import numpy as np;
import pandas as pd;
import copy;
import matplotlib.pyplot as plt;

In [6]:
def func(k,x):
    global act_fun;
    if act_fun[k] == 1:
        return relu(x);
    if act_fun[k] == 0:
        return x;

def gra_func(k,x):
    global act_fun;
    if act_fun[k] == 1:
        return gra_relu(x);
    if act_fun[k] == 0:
        return 1;

def relu(x):
    res = np.abs(x);
    res = (res + x)/2;
    return res;

def gra_relu(x):
    res = np.abs(x);
    res = np.sign(res + x);
    return res;
    

In [17]:
# 随机打乱训练集，从训练集中划分出测试集
def random_division(x,y=None,test_rate = 0):
    x = np.array(x);
    # 数据总长度
    n = np.shape(x)[0];
    # 生成下表序列并随即打乱
    sequence = np.array(range(n));
    np.random.shuffle(sequence);
    # 测试集长度
    n_test = int(np.ceil(n * test_rate));
    # 测试集序列与训练集序列
    se_test = sequence[:n_test];
    se_train = sequence[n_test:];
    # 训练集
    train_data = x[se_train];
    # 测试集
    test_data = x[se_test];
    # 有监督学习的情况
    if not type(y) == type(None):
        y = np.array(y);
        train_label = y[se_train];
        test_label = y[se_test];
        if not test_rate == 0:
            return (train_data,train_label,test_data,test_label);
        else:
            return (train_data,train_label);
    # 无监督情况
    if not test_rate == 0:
        return (train_data,test_data);
    else:
        return (train_data);

In [8]:
names_to_label = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2};
label_to_names = {value: key for key, value in names_to_label.items()}
df = pd.read_csv('iris.data', header=None)
xs = df.iloc[:, :4].values
ts = np.array([names_to_label[name] for name in df.iloc[:, -1]])
ys = np.zeros((ts.shape[0], 3));
ts = ts.reshape(-1,1);

In [9]:
# 对数据的归一化处理
max_x = np.max(np.abs(xs),axis=0);
xs = xs / max_x;

In [10]:
layers = 5;
nets = [16,64,256,32,1];
in_shape = [4] + nets[:-1];
out_shape = nets;
act_fun = [1,1,1,1,0];
x0 = xs;
n = x0.shape[0];
m = x0.shape[1];

In [11]:
w = [];
b = [];
x = [];
y = [];
delta = [];
for i in range(layers):
    wk = np.random.randn(in_shape[i],out_shape[i]);
    bk = np.random.randn(1,nets[i]);
    w.append(wk);
    b.append(bk);
    x.append([]);
    y.append([]);
    delta.append([]);

In [12]:
# 将初始化参数保存，以便控制变量分析
w_init = copy.deepcopy(w);
b_init = copy.deepcopy(b);
x_init = copy.deepcopy(x);
y_init = copy.deepcopy(y);
delta_init = copy.deepcopy(delta);

In [13]:
# 加载已保存的初始化参数
w = copy.deepcopy(w_init);
b = copy.deepcopy(b_init);
x = copy.deepcopy(x_init);
y = copy.deepcopy(y_init);
delta = copy.deepcopy(delta_init);

In [14]:
epoches = 10;
rate = 1e-6;
res_w = [];
res_b = [];
res_delta = [];
res_dw = [];
res_db = [];
for p in range(epoches):
    x0 = xs;
    # 正向传播
    # 输入层情况
    x0 = xs;
    x[0] = np.dot(x0,w[0]) + b[0];
    y[0] = func(1,x[0]);
    # 隐藏层
    for k in range(1,layers):
        x[k] = np.dot(y[k-1],w[k]) + b[k];
        y[k] = func(k-1,x[k]);
    loss = 1/x0.shape[0] * 1/2*np.sum((y[k] - ts)**2);
    # 反向传播
    res_w.append([]);
    res_b.append([]);
    res_dw.append([]);
    res_db.append([]);
    res_delta.append([]);
    # 输出层情况
    delta[k] = (y[k] - ts)*gra_func(k-1,x[k]);
    # delta量的归一化处理
    delta[k] = delta[k] / np.max(np.abs(delta[k]));
    db = np.sum(delta[k],axis=0).reshape(1,-1);
    dw = np.dot(y[k-1].T,delta[k]);
    res_w[p].append(copy.deepcopy(w[k]));
    res_b[p].append(copy.deepcopy(b[k]));
    res_delta[p].append(copy.deepcopy(delta[k]));
    res_dw[p].append(copy.deepcopy(dw));
    res_db[p].append(copy.deepcopy(db));
    b[k] -= rate*db;
    w[k] -= rate*dw;
    
    k -= 1;
    # 隐藏层
    while(k>0):
        delta[k] = np.dot(delta[k+1],w[k+1].T) * gra_func(k-1,x[k]);
        # delta的归一化处理
        delta[k] = delta[k] / np.max(np.abs(delta[k]));
        db = np.sum(delta[k],axis = 0).reshape(1,-1);
        dw = np.dot(y[k-1].T,delta[k]);
        res_w[p].append(copy.deepcopy(w[k]));
        res_b[p].append(copy.deepcopy(b[k]));
        res_delta[p].append(copy.deepcopy(delta[k]));
        res_dw[p].append(copy.deepcopy(dw));
        res_db[p].append(copy.deepcopy(db));
        b[k] -= rate*db;
        w[k] -= rate*dw;
        k -= 1;
    print('ep {}: loss: {:.2f}'.format(p,loss))

ep 0: loss: 30272.77
ep 1: loss: 23889.55
ep 2: loss: 18851.82
ep 3: loss: 14953.10
ep 4: loss: 11971.20
ep 5: loss: 9708.85
ep 6: loss: 7982.75
ep 7: loss: 6642.27
ep 8: loss: 5574.75
ep 9: loss: 4713.84


In [15]:
# 提取统计信息
data = res_dw; # 在这里修改需要输出的变量
abs_max = [];
mean_data = [];
med_data = [];
abs_min = [];
for i in range(len(data)):
    abs_max.append([]);
    mean_data.append([]);
    med_data.append([]);
    abs_min.append([]);

    for k in range(len(data[i])-1,-1,-1):
        abs_max[i].append(np.max(np.abs(data[i][k])));
        mean_data[i].append(np.mean(data[i][k]));
        med_data[i].append(np.median(data[i][k]));
        abs_min[i].append(np.min(np.abs(data[i][k])));

In [16]:
# 将统计信息输出到excel
data_pd = pd.DataFrame(abs_max);
writer = pd.ExcelWriter('abs_max.xlsx');
data_pd.to_excel(writer,'page_1',float_format='%.6f');
writer.save();
writer.close();

data_pd = pd.DataFrame(mean_data);
writer = pd.ExcelWriter('mean_data.xlsx');
data_pd.to_excel(writer,'page_1',float_format='%.6f');
writer.save();
writer.close();

data_pd = pd.DataFrame(med_data);
writer = pd.ExcelWriter('med_data.xlsx');
data_pd.to_excel(writer,'page_1',float_format='%.6f');
writer.save();
writer.close();

data_pd = pd.DataFrame(abs_min);
writer = pd.ExcelWriter('abs_min.xlsx');
data_pd.to_excel(writer,'page_1',float_format='%.6f');
writer.save();
writer.close();

ModuleNotFoundError: No module named 'openpyxl'